In [1]:
import ROOT

Welcome to JupyROOT 6.14/00


Getting ROOT, Python3, and Jupyter happy together on macOS was hard enough. Just going to grab HEPvector from disk. Don't do this in your code, just use pip to install.

In [2]:
import sys
sys.path.insert(0, '..')

In [3]:
from hepvector import Vector, Vector2D, Vector3D, LorentzVector
import numpy as np

We need a way to compare a hepvector to a ROOT vector:

In [4]:
def compare(npvec, rvec):
    rveclist = [rvec.X(), rvec.Y()]
    if hasattr(rvec, 'Z'):
        rveclist.append(rvec.Z())
    if hasattr(rvec, 'T'):
        rveclist.append(rvec.T())
        
    rvecarr = np.array(rveclist, np.double)
    return np.allclose(npvec, rvecarr)

# 2D vector comparison

In [5]:
v = Vector2D(1,2)
v2 = Vector2D(.3, .1)
r = ROOT.TVector2(1,2)
r2 = ROOT.TVector2(.3, .1)

In [6]:
assert v.x == r.X()
assert v.y == r.Y()

assert v[0] == r.X()
assert v[1] == r.Y()

In [7]:
assert v.phi() == r.Phi()
assert v.rho() == r.Mod()
assert v.mag() == r.Mod()
assert v.mag2() == r.Mod2()

In [8]:
assert compare(v.unit(), r.Unit())
assert np.isclose(v.angle(v2), r.DeltaPhi(r2))
# r.Norm(r2)
# r.Ort()
# r.Proj(r2)
# r.Rotate(phi)
# v.pt()
# v.pt2()

# 3D vector comparison

In [9]:
v = Vector3D(1,2,3)
v1 = Vector3D(.4,-.1,.9)
r = ROOT.TVector3(1,2,3)
r1 = ROOT.TVector3(.4,-.1,.9)

In [10]:
assert v.x == r.X()
assert v.y == r.Y()
assert v.z == r.Z()

In [11]:
assert v.mag() == r.Mag()       # get magnitude (=rho=Sqrt(x*x+y*y+z*z)))
assert v.mag2() == r.Mag2()     # get magnitude squared
assert v.theta() == r.Theta()   # get polar angle
assert np.cos(v.theta()) == r.CosTheta() # get cos of theta
assert v.phi() == r.Phi()       # get azimuth angle
assert v.pt() == r.Perp()       # get transverse component
assert v.pt2() == r.Perp2()     # get transverse component**2

In [12]:
# v.Perp(v1);
# v.Perp2(v1);

In [13]:
# v.PseudoRapidity();
assert compare(v.cross(v1), r.Cross(r1))
assert v.dot(v1) == r.Dot(r1)
assert compare(v + v1, r + r1)
assert compare(v - v1, r - r1)

# r.Rotate* are oddly only inplace
# v.rotate_euler(phi=30)

# Lorentz vector comparison

In [14]:
v = LorentzVector(1,2,3,.2)
v1 = LorentzVector(.4,.1,-.7,2)
r = ROOT.TLorentzVector(1,2,3,.2)
r1 = ROOT.TLorentzVector(.4,.1,-.7,2)

In [15]:
assert v.x == r.X()
assert v.y == r.Y()
assert v.z == r.Z()
assert v.t == r.T()
assert v.e() == r.E()

In [16]:
assert v.beta() == r.Beta()
assert v.dot(v1) == r.Dot(r1)
assert compare(v + v1, r + r1)
assert compare(v - v1, r - r1)
assert v.pt() == r.Perp()
assert v.pt2() == r.Perp2()
assert v.pseudorapidity() == r.PseudoRapidity()
assert compare(v.boost_vector(), r.BoostVector())
assert v.delta_r(v1) ==  r.DeltaR(r1)
assert v.eta() == r.Eta()
assert v1.rapidity() == r1.Rapidity()

# Defined methods

Let's look at all methods available (classmethods too):

In [17]:
import inspect
from itertools import zip_longest

ignore = {x for x,y in inspect.getmembers(np.ndarray)}
ignore |= {'__module__', '_repr_html_', '__slots__'}

insp = lambda cls: {x for x,y in inspect.getmembers(cls)} - ignore

v0 = insp(Vector)
v2 = insp(Vector2D) - v0
v3 = insp(Vector3D) - v2
l4 = insp(LorentzVector) - v3

mc = lambda x: max(max(map(len, x)), 8)

a,b,c,d = 'Vector Vector2D Vector3D LorentzVector'.split()
print(f'{a:{mc(v0)}} | {b:{mc(v2)}} | {c:{mc(v3)}} | {d:{mc(l4)}}')
print(f'{"":-^{mc(v0)}}-|-{"":-^{mc(v2)}}-|-{"":-^{mc(v3)}}-|-{"":-^{mc(l4)}}')
for a,b,c,d in zip_longest(v0, v2, v3, l4, fillvalue=''):
    print(f'{a:{mc(v0)}} | {b:{mc(v2)}} | {c:{mc(v3)}} | {d:{mc(l4)}}')

Vector      | Vector2D | Vector3D                | LorentzVector    
------------|----------|-------------------------|------------------
from_vector | pt2      | from_vector             | METRIC           
from_pandas | phi      | in_basis                | p                
to_pd       | X        | angle                   | rho              
mag2        | y        | theta                   | from_pt_eta_phi  
unit        | x        | Z                       | delta_r          
dims        | rho      | from_spherical_coords   | X                
origin      | Y        | r                       | y                
angle       | NAMES    | mag2                    | boost_vector     
mag         | pt       | dims                    | boost            
            |          | from_cylindrical_coords | Y                
            |          | rotate_axis             | NAMES            
            |          | origin                  | pt               
            |          | from_pand